# STFT Analysis/Synthesis - MusicBricks Tutorial

## Introduction

This tutorial will guide you through some tools for performing spectral analysis and synthesis using the Essentia library (http://www.essentia.upf.edu). STFT stands for Short-Time Fourier Transform and it processes an input audio signal as a sequence of spectral frames. Spectral frames are complex-valued arrays contain the frequency representation of the windowed input signal.

This algorithm shows how to analyze the input signal, and resynthesize it again, allowing to apply new transformations directly on the spectral domain.

You should first install the Essentia library with Python bindings. Installation instructions are detailed here: http://essentia.upf.edu/documentation/installing.html . 


## Processing steps

In [1]:
# import essentia in streaming mode
import essentia
import essentia.streaming as es

Define the parameters of the STFT workflow

In [2]:
# algorithm parameters
framesize = 1024
hopsize = 256

Specify input and output audio filenames

In [3]:
inputFilename = 'singing-female.wav'
outputFilename = 'singing-female-stft.wav'

In [4]:
# create an audio loader and import audio file
loader = es.MonoLoader(filename=inputFilename, sampleRate=44100)
pool = essentia.Pool()

Define algorithm chain for frame-by-frame process: 
FrameCutter -> Windowing -> FFT -> IFFT -> OverlapAdd -> AudioWriter

In [5]:
# algorithm instantation
fcut = es.FrameCutter(frameSize=framesize, hopSize=hopsize, startFromZero=False)
w = es.Windowing(type="hann")
fft = es.FFT(size=framesize)
ifft = es.IFFT(size=framesize)
overl = es.OverlapAdd(frameSize=framesize, hopSize=hopsize, gain=1./framesize )
awrite = es.MonoWriter(filename=outputFilename, sampleRate=44100)

Now we set the algorithm network and store the processed audio samples in the output file

In [6]:
loader.audio >> fcut.signal
fcut.frame >> w.frame
w.frame >> fft.frame
fft.fft >> ifft.fft
ifft.frame >> overl.frame
overl.signal >> awrite.audio
overl.signal >> (pool, 'audio')

Finally we run the process that will store an output file in a WAV file

In [7]:
essentia.run(loader)

[wav @ 0x55df478c8400] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x55df478c8400] Encoder did not produce proper pts, making some up.
